In [1]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier

In [2]:
import torch
import torch.nn as nn

In [3]:
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [4]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset(vectors = 
                                                                                       GLOVE_EMBEDDING)

In [5]:
batch_size = 32
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=False, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=False, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=False, repeat=False)

In [6]:
val_list = list(val_iter)

In [7]:
batch = val_list[600]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {}".format(' '.join([train_dataset.fields['review_text'].vocab.itos[_] for _ in x[idx]]),
         train_dataset.fields['theme'].vocab.itos[y[idx]]))

<sos> as usual sad true story ... but great acting <eos> | plot
<sos> anything with jennifer lawrence is worth your time . <eos> | other
<sos> you 'll find them in `` dirty grandpa . <eos> | other
<sos> no direction , worthwhile plot , or acting . <eos> | plot
<sos> although be warned there are some gory scenes . <eos> | plot
<sos> one of the best musical movie for me . <eos> | other
<sos> so disappointed in jake gyllenhall for this one . <eos> | other
<sos> it takes much more than a balance to walk <eos> | other
<sos> that highlights the greatest of a true thriller . <eos> | other
<sos> overall , though , it left me cold . <eos> | other


In [8]:
#review_text_FIELD.vocab.vectors.shape, len(review_text_FIELD.vocab.itos)

In [9]:
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [10]:
vocab_size = review_text_FIELD.vocab.vectors.shape[0]
label_size = len(theme_FIELD.vocab) - 1
emb_dim = review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 500
layers = 2
dropout = .2

vocab_size, label_size, emb_dim, vectors.shape

(30004, 5, 300, torch.Size([30004, 300]))

In [99]:
class BaseModel(nn.Module):

    def __init__(self, 
                 ninp = vocab_size, 
                 linp = label_size, 
                 emb_dim = emb_dim, 
                 emb_lab = 20,
                 nhid = hidden_dim, 
                 nout = vocab_size, 
                 nlayers = layers, 
                 dropout = dropout, 
                 vectors = vectors,
                 pretrained = False):
        super().__init__()
        
        self.ninp = ninp
        self.linp = linp
        self.emb_dim = emb_dim
        self.emb_lab = emb_lab
        self.nhid = nhid
        self.nout = nout
        self.nlayers = nlayers
        self.drop = nn.Dropout(dropout)

        self.word_embedding = nn.Embedding(ninp, emb_dim)
        self.label_embedding = nn.Embedding(linp, emb_lab)
        
        self.rnn = nn.LSTM(emb_dim , nhid, nlayers, dropout=dropout) #+ emb_lab
        self.rnn.flatten_parameters()
        self.decoder = nn.Linear(nhid, nout)
        self.softmax = nn.Softmax(dim=-1)

        if pretrained:
            self.word_embedding.weight.data.copy_(vectors)
            
        self.init_weights()    
            
    def init_weights(self):
        initrange = .1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, reviews, labels, hidden):
        R = self.word_embedding(reviews)
        #L = self.label_embedding(labels)
        #L = torch.cat([L.unsqueeze(0)]*R.shape[0])
        #X = torch.cat([R, L], -1)
        X = R
        
        X = self.drop(X)
        X, hidden = self.rnn(X, hidden)
        X = self.drop(X)
        
        X = X.view(X.size(0)*X.size(1), X.size(2))
        
        X = self.decoder(X)
        log_probs = self.softmax(X)
        return log_probs, hidden
      

In [100]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [101]:
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss_e = 0
    total_number_of_words = 0

    with torch.no_grad():
        for i, batch in enumerate(data_source):
            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 2:
                data, targets = batch[:-1,:], batch[1:,:]
                number_of_words = data.shape[0] * data.shape[1]
                
                output, hidden = model(data, labels, hidden)
                output_flat = output.contiguous().view(-1, vocab_size)

                total_loss_e += criterion(output_flat, targets.contiguous().view(-1)).data.float()
                total_number_of_words += number_of_words
                hidden = repackage_hidden(hidden)
            
    return total_loss_e.item() / total_number_of_words

In [102]:
import numpy as np
import gc

In [103]:
def train(model, ep0, epN, train_iter, dev_iter, optimizer, criterion, 
          max_grad_norm, model_name, best_ppl = float('inf')):
    
    best_ppl = best_ppl
    
    len_train_iter = len(train_iter)
    for epoch in range(ep0, epN):
        model.train()
        total_loss_e = 0
        total_number_of_words = 0 
        
        for i, batch in enumerate(train_iter):

            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            if batch.shape[0] > 2:
                data, targets = batch[:-1,:], batch[1:,:]
                
                number_of_words = data.shape[0] * data.shape[1]
                
                output, hidden = model(data, labels, hidden)
                hidden = repackage_hidden(hidden)

                output_flat = output.contiguous().view(-1, vocab_size)
                epoch_loss = criterion(output_flat, targets.contiguous().view(-1))
                total_loss_e += epoch_loss.data.float()
                total_number_of_words += number_of_words
            
                torch.nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)
                optimizer.zero_grad()
                epoch_loss.backward()
                optimizer.step()

                if i % 500 == 0:
                    cur_loss = epoch_loss.item() / number_of_words
                    tr_ppl_print = np.exp(cur_loss)
                    print("| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {}".format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3)))

                #gc.collect()
                #torch.cuda.empty_cache()
                
                if i % 2000 == 0:
                    cur_loss = total_loss_e.item() / total_number_of_words
                    tr_ppl_print = np.exp(cur_loss)
                    val_loss_eval = evaluate(dev_iter)
                    val_ppl_print = np.exp(val_loss_eval)
                    
                    template = "| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {} | val_loss {} | val_ppl {}"
                    print(template.format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3), 
                            np.round(val_loss_eval, 3), np.round(val_ppl_print, 3)))

                    if val_ppl_print < best_ppl :
                        print('old best ppl {} new best ppl {}'.format(best_ppl, val_ppl_print))
                        best_ppl = val_ppl_print
                        best_model_name = 'best_model_{}_{}.model'.format(model_name, best_ppl)
                        print('save model...', best_model_name)
                        with open(best_model_name, 'wb') as file:
                            torch.save(model, file) 

                    gc.collect()
                    model.train()


In [104]:
#with open('./best_model_base_model_ppl_250.3359726270283.model', 'rb') as file:
#    model = torch.load(file)#BaseModel().cuda()
#    model.eval()

In [105]:
model = BaseModel(pretrained=True).cuda()

In [106]:
model

BaseModel(
  (drop): Dropout(p=0.2)
  (word_embedding): Embedding(30004, 300)
  (label_embedding): Embedding(5, 20)
  (rnn): LSTM(300, 500, num_layers=2, dropout=0.2)
  (decoder): Linear(in_features=500, out_features=30004, bias=True)
  (softmax): Softmax()
)

In [107]:
# implement PPL
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction='sum', 
                       ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train(model,
      ep0 = 0,
      epN = 1,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'base_model_ppl',
      best_ppl = float('inf'))

| epoch   0 | batch 500 / 21962 | train_loss 10.051 | train_ppl 23187.812
| epoch   0 | batch 1000 / 21962 | train_loss 10.09 | train_ppl 24110.268
| epoch   0 | batch 1500 / 21962 | train_loss 10.137 | train_ppl 25267.456
| epoch   0 | batch 2000 / 21962 | train_loss 10.142 | train_ppl 25399.287
| epoch   0 | batch 2000 / 21962 | train_loss 10.079 | train_ppl 23834.063 | val_loss 10.217 | val_ppl 27351.027
old best ppl inf new best ppl 27351.026519136412
save model... best_model_base_model_ppl_27351.026519136412.model


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BaseModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   0 | batch 2500 / 21962 | train_loss 10.14 | train_ppl 25323.917
| epoch   0 | batch 3000 / 21962 | train_loss 10.165 | train_ppl 25967.78
| epoch   0 | batch 3500 / 21962 | train_loss 10.153 | train_ppl 25665.246
| epoch   0 | batch 4000 / 21962 | train_loss 10.156 | train_ppl 25754.451
| epoch   0 | batch 4000 / 21962 | train_loss 10.125 | train_ppl 24946.977 | val_loss 10.206 | val_ppl 27052.439
old best ppl 27351.026519136412 new best ppl 27052.43865718559
save model... best_model_base_model_ppl_27052.43865718559.model
| epoch   0 | batch 4500 / 21962 | train_loss 10.106 | train_ppl 24490.528
| epoch   0 | batch 5000 / 21962 | train_loss 10.065 | train_ppl 23515.18
| epoch   0 | batch 5500 / 21962 | train_loss 10.099 | train_ppl 24316.985
| epoch   0 | batch 6000 / 21962 | train_loss 10.115 | train_ppl 24709.99
| epoch   0 | batch 6000 / 21962 | train_loss 10.118 | train_ppl 24776.903 | val_loss 10.205 | val_ppl 27028.928
old best ppl 27052.43865718559 new best ppl 27028.9